<a href="https://colab.research.google.com/github/biodatlab/score-claim-extraction/blob/main/Claim_Extraction_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradio App for Claim Extraction

This notebook deploys a simple gradio app to perform claim extraction on an abstract using our model.

In [ ]:
!pip install gradio
!pip install transformers==4.28.0

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import torch

model_name = 'biodatlab/score-claim-identification'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def check_claim(line):
  inputs = tokenizer(line, return_tensors='pt',truncation=True)
  with torch.no_grad():
    logits = model(**inputs).logits
  predicted_class_id = logits.argmax().item()


In [ ]:
def inference(abstract):
  claims = []
  lines = abstract.split('. ')
  # for line in lines:
  #   if check_claim(line):
  #     return line
  inputs = tokenizer(lines,return_tensors='pt',truncation=True,padding="longest")
  output = model(**inputs).logits
  for (i,out) in enumerate(output):
    pred = out.argmax().item()
    if pred:
      claims.append(lines[i])
  if claims:
    return '.\n'.join(claims)
  return 'No claims were made here'

In [ ]:
abst = """Research has shown that people often reinterpret their experiences of others' harm and suffering to maintain the functional belief that people get what they deserve (e.g., by blaming the victim). Rather than focusing on such reactive responses to harm and suffering, across 7 studies we examined whether people selectively and proactively choose to be exposed to information about deserved rather than undeserved outcomes. We consistently found that participants selectively chose to learn that bad (good) things happened to bad (good) people (Studies 1 to 7) that is, they selectively exposed themselves to deserved outcomes. This effect was mediated by the perceived deservingness of outcomes (Studies 2 and 3), and was reduced when participants learned that wrongdoers otherwise received "just deserts" for their transgressions (Study 7). Participants were not simply selectively avoiding information about undeserved outcomes but actively sought ihformation about deserved outcomes (Studies 3 and 4), and participants invested effort in this pattern of selective exposure, seeking out information about deserved outcomes even when it was more time-consuming to find than undeserved outcomes (Studies 5 and 6). Taken together, these findings cast light on a more proactive, anticipatory means by which people maintain a commitment to deservingness. (C) 2016 Elsevier Inc. All rights reserved."""
inference(abst)

In [ ]:
import gradio as gr

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# Claim Identification")
    gr.Markdown("This demo finds claims made by an abstract.")
    abst = gr.Textbox(label="Abstract")
    greet_btn = gr.Button("Find Claims")
    output = gr.Textbox(label="Detected Claims")
    greet_btn.click(fn=inference, inputs=abst, outputs=output, api_name="inference")
    examples = gr.Examples(examples=["This study adopted a person (actor) by partner perspective to examine how actor personality traits, partner personality traits, and specific actor by partner personality trait interactions predict actor's depressive symptoms across the first 2years of the transition to parenthood. Data were collected from a large sample of new parents (both partners in each couple) 6weeks before the birth of their first child, and then at 6, 12, 18, and 24months postpartum. The results revealed that higher actor neuroticism and lower partner agreeableness predicted higher levels of depressive symptoms in actors. Moreover, the specific combination of high actor neuroticism and low partner agreeableness was a particularly problematic combination, which was intensified when prepartum dysfunctional problem-solving communication and aggression existed in the relationship. These results demonstrate the importance of considering certain actor by partner disposition pairings to better understand actors' emotional well-being during major life transitions."],inputs=[abst])

demo.launch(share=True,debug=True)